In [49]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

In [50]:
data = pd.read_csv('Iris.csv')
data[50:60]

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
50,51,7.0,3.2,4.7,1.4,Iris-versicolor
51,52,6.4,3.2,4.5,1.5,Iris-versicolor
52,53,6.9,3.1,4.9,1.5,Iris-versicolor
53,54,5.5,2.3,4.0,1.3,Iris-versicolor
54,55,6.5,2.8,4.6,1.5,Iris-versicolor
55,56,5.7,2.8,4.5,1.3,Iris-versicolor
56,57,6.3,3.3,4.7,1.6,Iris-versicolor
57,58,4.9,2.4,3.3,1.0,Iris-versicolor
58,59,6.6,2.9,4.6,1.3,Iris-versicolor
59,60,5.2,2.7,3.9,1.4,Iris-versicolor


In [51]:
X_features=['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']
Y_label=['Species']

In [52]:
X=data[X_features]
Y=data[Y_label]

encoder=LabelEncoder()
y1=encoder.fit_transform(Y)
Y=pd.get_dummies(y1).values 

/opt/tljh/user/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [53]:
# Train , test split using sklearn

train_X, test_X, train_Y, test_Y=train_test_split(X,Y, test_size=0.2, random_state=0)

In [54]:
l0 = tf.keras.layers.Dense(units=4, input_shape=(4,), activation='relu')
l1 = tf.keras.layers.Dense(units=3, activation='softmax')
model = tf.keras.Sequential([l0,l1])

In [55]:
# Model compiling

model.compile(tf.keras.optimizers.Adam(0.02), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 15        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.fit(train_X, train_Y, epochs=100)

Epoch 1/100
4/4 [==============================] - 0s 1ms/step - loss: 4.0389 - accuracy: 0.3250
Epoch 2/100
4/4 [==============================] - 0s 936us/step - loss: 2.6914 - accuracy: 0.5917
Epoch 3/100
4/4 [==============================] - 0s 933us/step - loss: 1.8158 - accuracy: 0.4917
Epoch 4/100
4/4 [==============================] - 0s 841us/step - loss: 1.1864 - accuracy: 0.5333
Epoch 5/100
4/4 [==============================] - 0s 912us/step - loss: 1.0486 - accuracy: 0.4167
Epoch 6/100
4/4 [==============================] - 0s 921us/step - loss: 1.0756 - accuracy: 0.3667
Epoch 7/100
4/4 [==============================] - 0s 880us/step - loss: 1.1006 - accuracy: 0.3667
Epoch 8/100
4/4 [==============================] - 0s 1ms/step - loss: 1.0995 - accuracy: 0.3667
Epoch 9/100
4/4 [==============================] - 0s 955us/step - loss: 1.0819 - accuracy: 0.3667
Epoch 10/100
4/4 [==============================] - 0s 832us/step - loss: 1.0515 - accuracy: 0.3750
Epoch 11/100


In [57]:
loss, acc = model.evaluate(test_X, test_Y)
print(type(test_X))
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
print("Restored model, loss: {}".format(loss))

1/1 [==============================] - 0s 1ms/step - loss: 0.1692 - accuracy: 1.0000
<class 'pandas.core.frame.DataFrame'>
Restored model, accuracy: 100.00%
Restored model, loss: 0.16915734112262726


In [58]:
#Save model in TF PB format
saved_model_dir = 'Iris_model.TF'
model.save(saved_model_dir)

INFO:tensorflow:Assets written to: Iris_model.TF/assets


In [59]:
#make it executable file
!deepCC Iris_model.TF --format=tensorflow

reading [tensorflow model] from 'Iris_model.TF'
Saved 'Iris_model.TF.onnx'
reading onnx model from file  Iris_model.TF.onnx
Model info:
  ir_vesion :  4 
  doc       : 
WARN (ONNX): terminal (input/output) dense_2_input_0's shape is less than 1.
             changing it to 1.
WARN (ONNX): terminal (input/output) Identity_0's shape is less than 1.
             changing it to 1.
running DNNC graph sanity check ... passed.
Writing C++ file  Iris_model.TF_deepC/Iris_model.TF.cpp
INFO (ONNX): model files are ready in dir Iris_model.TF_deepC
g++ -O3 -I. -I/opt/tljh/user/lib/python3.7/site-packages/deepC-0.13-py3.7-linux-x86_64.egg/deepC/include -isystem /opt/tljh/user/lib/python3.7/site-packages/deepC-0.13-py3.7-linux-x86_64.egg/deepC/packages/eigen-eigen-323c052e1731 Iris_model.TF_deepC/Iris_model.TF.cpp -o Iris_model.TF_deepC/Iris_model.TF.exe
Model executable  Iris_model.TF_deepC/Iris_model.TF.exe


In [60]:
#example=iris_data.data[3]
#print(example)
!./Iris_model.TF_deepC/Iris_model.TF.exe iris_test.data

reading file iris_test.data.
writing file Identity_0.out.


In [61]:
#print(test_iris.data)
nn_out = np.loadtxt('Identity_0.out')
print ("Model prediction is Class :", np.argmax(nn_out),"for the iris_test.data")

Model prediction is Class : 1 for the iris_test.data
